In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
pwd

'/content/gdrive/My Drive/Kaggle'

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from sklearn.impute import SimpleImputer
import fbprophet
from itertools import cycle
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import mean_squared_error
from keras.layers import BatchNormalization
from keras.layers import LSTM


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
calendar= pd.read_csv('/content/gdrive/My Drive/Kaggle/calendar.csv')
prices= pd.read_csv('/content/gdrive/My Drive/Kaggle/sell_prices.csv')
sales_train_evaluation= pd.read_csv('/content/gdrive/My Drive/Kaggle/sales_train_evaluation.csv')

In [ ]:
print("Calendar shape: ", calendar.shape)
print("prices shape: ",prices.shape)
print("sales shape : ", sales_train_evaluation.shape)
#print("sales validation shape: " , sales_train_validation.shape)

Calendar shape:  (1969, 14)
prices shape:  (6841121, 4)
sales shape :  (30490, 1947)


In [ ]:
#Downcast in order to save memory
def transform (df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        '''elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')'''
    return df  

sales_train_evaluation = transform(sales_train_evaluation)
prices = transform(prices)


In [ ]:
prices = transform(prices)

In [ ]:
calendar.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


In [ ]:
prices.head(2)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58


In [ ]:
sales_train_evaluation.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,2,4,0,0,0,0,3,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Currently, the data is in three dataframes: sales, prices & calendar. The sales dataframe contains daily sales data with days(d_1 - d_1969) as columns. The prices dataframe contains items' price details and calendar contains data about the days d.

# Data Preprocessing

In [ ]:
# Preprocess: remove id, item_id, dept_id, cat_id, store_id, state_id columns
startDay = 350  # Remove the first 350 days in train sales data due to zero_inflated data
sales_train_evaluation = sales_train_evaluation.T
sales_train_evaluation = sales_train_evaluation[6 + startDay:]
sales_train_evaluation.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,0,1,0,4,17,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16,0,0,0,0,0,2,0,0,...,0,0,0,0,14,0,1,0,0,1,0,1,60,4,2,0,10,2,0,9,2,9,0,0,7,0,6,0,0,0,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,0,0,0,2,0,0,3,0,0,0,0,1,2,0,0,0,0,0,0,0,0,9,0,0,0,3,0,1,0,0,...,0,0,0,0,18,5,0,0,0,1,0,0,30,4,0,0,6,5,0,8,7,12,0,0,0,0,3,0,0,1,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,0,1,0,0,10,0,0,0,0,0,0,1,2,0,0,0,0,3,0,0,0,3,0,0,0,0,0,1,0,0,...,0,0,0,0,18,5,2,0,0,1,0,0,11,1,0,0,10,4,0,11,9,7,0,0,1,0,16,0,0,2,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,0,0,0,1,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,21,0,1,0,0,0,1,0,0,...,0,0,0,0,13,1,1,0,0,1,0,0,15,7,0,0,4,1,0,3,2,2,0,0,1,0,9,0,0,0,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,0,0,0,0,6,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,11,1,0,0,0,2,0,0,9,4,0,0,8,7,0,8,20,2,0,0,3,0,2,0,0,1,0,7,1,0,1,0,1,1,0,0


I have removed the first six colums id, item_id, dept_id, cat_id, store_id, state_id columns, to end up only days as rows

In [ ]:
# Initialize a dataframe with zeros for 1969 days in the calendar
 #"1" is assigned to the days before the event_name_. 
daysBeforeEvent1 = pd.DataFrame(np.zeros((1969,1)))
daysBeforeEvent2 = pd.DataFrame(np.zeros((1969,1)))

snap_CA = pd.DataFrame(np.zeros((1969,1)))
snap_TX = pd.DataFrame(np.zeros((1969,1)))
snap_WI = pd.DataFrame(np.zeros((1969,1)))


# Label 1 to one day before the event_name_1 
# Label 1 to one day before the event_name_2
# Sales are likely to increase one day before events like superbowl etc.

# Label 1 to days on snap_CA
# Label 1 to days on snap_TX
# Label 1 to days on snap_WI

for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent1[0][x-1] = 1 
            
    if((pd.isnull(calendar["event_name_2"][x])) == False):
           daysBeforeEvent2[0][x-1] = 1    
    
    
    if((pd.isnull(calendar["snap_CA"][x])) == False):
           snap_CA[0][x] = 1    
        
    if((pd.isnull(calendar["snap_TX"][x])) == False):
           snap_TX[0][x] = 1    
        
    if((pd.isnull(calendar["snap_WI"][x])) == False):
           snap_WI[0][x] = 1

In [ ]:
del calendar

**Creating training, validation, evaluation dataset**

In [ ]:
# split dataset into evaluation (last 2 weeks), validation (first 2 weeks), training  
# input for predicting validation period day 1941 to 1969

daysBeforeEvent1_eval = daysBeforeEvent1[1941:]
daysBeforeEvent2_eval = daysBeforeEvent2[1941:]

snap_CA_eval = snap_CA[1941:]
snap_TX_eval = snap_TX[1941:]
snap_WI_eval = snap_WI[1941:]


# input for predicting validation period day 1913 to 1941

daysBeforeEvent1_valid = daysBeforeEvent1[1913:1941] 
daysBeforeEvent2_valid = daysBeforeEvent2[1913:1941]

snap_CA_valid = snap_CA[1913:1941] 
snap_TX_valid = snap_TX[1913:1941]
snap_WI_valid = snap_WI[1913:1941]

# input for training as a feature
# daysBeforeEvent1 = daysBeforeEvent1[startDay:1913] 
# daysBeforeEvent2 = daysBeforeEvent2[startDay:1913] 
daysBeforeEvent1 = daysBeforeEvent1[startDay:1941] 
daysBeforeEvent2 = daysBeforeEvent2[startDay:1941]

snap_CA = snap_CA[startDay:1941] 
snap_TX = snap_TX[startDay:1941] 
snap_WI = snap_WI[startDay:1941] 

In [ ]:
#Before concatanation with our main data "sale_tratrain_evaluation", indexes are made same and column name is changed to "oneDayBeforeEvent"
daysBeforeEvent1.columns = ["oneDayBeforeEvent1"]
daysBeforeEvent1.index = sales_train_evaluation.index

daysBeforeEvent2.columns = ["oneDayBeforeEvent2"]
daysBeforeEvent2.index = sales_train_evaluation.index


snap_CA.columns = ["snap_CA"]
snap_CA.index = sales_train_evaluation.index

snap_TX.columns = ["snap_TX"]
snap_TX.index = sales_train_evaluation.index

snap_WI.columns = ["snap_WI"]
snap_WI.index = sales_train_evaluation.index

In [ ]:
sales_train_evaluation = pd.concat([sales_train_evaluation, daysBeforeEvent1, daysBeforeEvent2,
                        snap_CA, snap_TX, snap_WI], axis = 1, sort=False)
sales_train_evaluation.head()  # additional features (event1, event2, SNAP) are added

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489,oneDayBeforeEvent1,oneDayBeforeEvent2,snap_CA,snap_TX,snap_WI
d_351,0,0,0,2,0,0,0,24,3,2,0,1,0,4,17,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16,0,0,0,0,0,2,0,0,...,0,1,0,0,1,0,1,60,4,2,0,10,2,0,9,2,9,0,0,7,0,6,0,0,0,0,9,1,0,11,0,0,1,0,0,0.0,0.0,1.0,1.0,1.0
d_352,0,0,0,0,0,0,0,9,0,2,0,0,0,2,0,0,3,0,0,0,0,1,2,0,0,0,0,0,0,0,0,9,0,0,0,3,0,1,0,0,...,5,0,0,0,1,0,0,30,4,0,0,6,5,0,8,7,12,0,0,0,0,3,0,0,1,0,5,4,0,8,0,1,2,0,0,1.0,0.0,1.0,1.0,1.0
d_353,0,0,0,4,2,0,0,2,1,1,0,1,0,0,10,0,0,0,0,0,0,1,2,0,0,0,0,3,0,0,0,3,0,0,0,0,0,1,0,0,...,5,2,0,0,1,0,0,11,1,0,0,10,4,0,11,9,7,0,0,1,0,16,0,0,2,0,15,2,0,3,0,1,2,0,0,0.0,0.0,1.0,1.0,1.0
d_354,0,1,0,2,0,0,0,7,1,0,0,0,0,1,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,21,0,1,0,0,0,1,0,0,...,1,1,0,0,1,0,0,15,7,0,0,4,1,0,3,2,2,0,0,1,0,9,0,0,0,0,5,1,0,3,0,0,0,0,0,0.0,0.0,1.0,1.0,1.0
d_355,0,0,0,1,2,0,0,0,0,0,0,0,0,0,6,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,2,0,0,9,4,0,0,8,7,0,8,20,2,0,0,3,0,2,0,0,1,0,7,1,0,1,0,1,1,0,0,0.0,0.0,1.0,1.0,1.0


**Standardizing features**

This helps to ensure that the sales values are between 0 and 1, and this helps the gradient descent optimization algorithm in LSTM model.

In [ ]:
#Feature Scaling: Scale features using min-max scaler in range 0-1
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))

train_sales_scaled = sc.fit_transform(sales_train_evaluation)

In [ ]:
timesteps = 28  # use the last 28 days to predict the next day's sales
X_train = []
y_train = []

for i in range(timesteps, 1941 - startDay):
    X_train.append(train_sales_scaled[i-timesteps:i])
    y_train.append(train_sales_scaled[i][0:30490])

In [ ]:
del train_sales_scaled

In [ ]:
#Convert to np array to be able to feed the LSTM model,### Data preparation for LSTM, turn 2D array to 3D as input
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1563, 28, 30495)
(1563, 30490)


**CNN-LSTM Model**


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout

tf.random.set_seed(51)
np.random.seed(51)

n_timesteps = X_train.shape[1]
n_products = X_train.shape[2]


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=7,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=(n_timesteps, n_products)),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=64, kernel_size=7, 
                           strides=1, activation='relu', padding="causal"),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30490)
])


opt_adam = tf.keras.optimizers.Adam(clipvalue=0.5)

model.compile(loss='mse',
              optimizer=opt_adam, 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 28, 128)           27323648  
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 14, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 14, 64)            57408     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7, 64)             0         
_________________________________________________________________
bidirectional (Bidirectional (None, 7, 1024)           2363392   
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 1024)           4096      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 512)            2

In [ ]:
# Define a Callback class that stops training when the val_loss degrades above 150 epochs
# and also saves the model as checkpoints

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=32)

In [ ]:
# Fitting the RNN to the Training set
epochs = 45
batch_size = 68
model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/45
23/23 [==============================] - 3s 142ms/step - loss: 0.0257 - root_mean_squared_error: 0.1604
Epoch 2/45
23/23 [==============================] - 3s 128ms/step - loss: 0.0194 - root_mean_squared_error: 0.1393
Epoch 3/45
23/23 [==============================] - 3s 131ms/step - loss: 0.0161 - root_mean_squared_error: 0.1268
Epoch 4/45
23/23 [==============================] - 3s 130ms/step - loss: 0.0148 - root_mean_squared_error: 0.1216
Epoch 5/45
23/23 [==============================] - 3s 131ms/step - loss: 0.0146 - root_mean_squared_error: 0.1209
Epoch 6/45
23/23 [==============================] - 3s 130ms/step - loss: 0.0144 - root_mean_squared_error: 0.1202
Epoch 7/45
23/23 [==============================] - 3s 129ms/step - loss: 0.0143 - root_mean_squared_error: 0.1196
Epoch 8/45
23/23 [==============================] - 3s 130ms/step - loss: 0.0142 - root_mean_squared_error: 0.1191
Epoch 9/45
23/23 [==============================] - 3s 129ms/step - loss: 0.0141

* Take last days, 28 ("timestep" parameter) in order to predict firts unknown day's sales.
* Before using values for prediction, again use min-max transformation

In [ ]:
#alternate method for LSTM either execute upper one or this ome:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
layer_1_units=50
regressor.add(LSTM(units = layer_1_units, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
layer_3_units=400
regressor.add(LSTM(units = layer_3_units, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
layer_3_units=400
regressor.add(LSTM(units = layer_3_units))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 30490))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
epoch_no=32
batch_size_RNN=44
regressor.fit(X_train, y_train, epochs = epoch_no, batch_size = batch_size_RNN)

Epoch 1/32
36/36 [==============================] - 3s 74ms/step - loss: 0.0179
Epoch 2/32
36/36 [==============================] - 3s 73ms/step - loss: 0.0157
Epoch 3/32
36/36 [==============================] - 3s 81ms/step - loss: 0.0153
Epoch 4/32
36/36 [==============================] - 3s 76ms/step - loss: 0.0150
Epoch 5/32
36/36 [==============================] - 3s 79ms/step - loss: 0.0148
Epoch 6/32
36/36 [==============================] - 3s 75ms/step - loss: 0.0146
Epoch 7/32
36/36 [==============================] - 3s 74ms/step - loss: 0.0144
Epoch 8/32
36/36 [==============================] - 3s 76ms/step - loss: 0.0142
Epoch 9/32
36/36 [==============================] - 3s 73ms/step - loss: 0.0141
Epoch 10/32
36/36 [==============================] - 3s 78ms/step - loss: 0.0140
Epoch 11/32
36/36 [==============================] - 3s 74ms/step - loss: 0.0139
Epoch 12/32
36/36 [==============================] - 3s 79ms/step - loss: 0.0138
Epoch 13/32
36/36 [==================

In [ ]:
inputs_eval = sales_train_evaluation[-timesteps:]
inputs_eval = sc.transform(inputs_eval)

inputs = sales_train_evaluation[-timesteps*2:-timesteps]
inputs = sc.transform(inputs)

Predicting the future sales for validation and evaluation periods


In [ ]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
    predicted_stock_price = model.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30495))
    
    testInput = np.column_stack((np.array(predicted_stock_price),
                                 daysBeforeEvent1_valid.loc[1913 + j - timesteps],
                                 daysBeforeEvent2_valid.loc[1913 + j - timesteps],
                                 snap_CA_valid.loc[1913 + j - timesteps],
                                snap_TX_valid.loc[1913 + j - timesteps],
                                snap_WI_valid.loc[1913 + j - timesteps]))

    X_test = np.append(X_test, testInput).reshape(1,j + 1,30495)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)

In [ ]:
X_eval = []
X_eval.append(inputs_eval[0:timesteps])
X_eval = np.array(X_eval)
predictions_eval = []

for j in range(timesteps,timesteps + 28):
    predicted_stock_price = model.predict(X_eval[0,j - timesteps:j].reshape(1, timesteps, 30495))
    
    testInput = np.column_stack((np.array(predicted_stock_price),
                                 daysBeforeEvent1_eval.loc[1941 + j - timesteps],
                                 daysBeforeEvent2_eval.loc[1941 + j - timesteps],
                                 snap_CA_eval.loc[1941 + j - timesteps],
                                snap_TX_eval.loc[1941 + j - timesteps],
                                snap_WI_eval.loc[1941 + j - timesteps]))

    X_eval = np.append(X_eval, testInput).reshape(1,j + 1,30495)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions_eval.append(predicted_stock_price)

In [ ]:
print(predictions_eval)

[array([[1.04071349, 0.13319814, 0.74403271, ..., 0.59296631, 1.67872238,
        1.47395146]]), array([[1.04400739, 0.14682303, 0.66828176, ..., 0.50846228, 1.58630669,
        1.09907645]]), array([[0.95177934, 0.19544285, 0.56959516, ..., 0.59079738, 1.3714681 ,
        1.07220501]]), array([[0.90204224, 0.2143088 , 0.46656369, ..., 0.545904  , 1.20334974,
        0.96587169]]), array([[0.98981537, 0.06132847, 0.72623129, ..., 0.49514187, 1.14666626,
        1.02491176]]), array([[ 1.57014877, -0.21924626,  0.99443731, ...,  0.51518087,
         1.46687883,  2.39389783]]), array([[1.14112824, 0.08822239, 0.90535989, ..., 0.79641019, 1.56725478,
        1.25786179]]), array([[0.87420031, 0.11549883, 0.69849247, ..., 0.86186912, 1.67817378,
        0.98344427]]), array([[0.92105657, 0.15005157, 0.67929858, ..., 0.87227683, 1.49669552,
        0.97369945]]), array([[0.96267648, 0.14722593, 0.58870482, ..., 0.83734727, 1.54952109,
        0.86956036]]), array([[0.86420417, 0.12749324, 0

In [ ]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))
submission = submission.T

submission_eval = pd.DataFrame(data=np.array(predictions_eval).reshape(28,30490))
submission_eval = submission_eval.T

submission = pd.concat((submission, submission_eval), ignore_index=True)

sample_submission = pd.read_csv("/content/gdrive/My Drive/Kaggle/sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

submission[cols] = submission[cols].mask(submission[cols] < 0, 0)
submission.to_csv("submission.csv", index=False)

In [ ]:
submission_df =pd.read_csv("/content/gdrive/My Drive/Kaggle/submission.csv")

In [ ]:
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.735134,0.797842,0.889317,0.875289,0.996212,2.005056,1.520738,0.824752,0.838771,0.927039,0.913499,1.043305,2.557063,1.198605,1.032555,1.033830,1.147260,1.124510,1.332506,1.833110,1.200897,1.100360,1.143346,1.062272,1.226447,1.194096,1.393663,0.966278
1,HOBBIES_1_002_CA_1_validation,0.018029,0.119322,0.185917,0.113565,0.000000,0.000000,0.269530,0.028477,0.083741,0.104895,0.077125,0.000000,0.000000,0.000000,0.000000,0.066468,0.090620,0.161787,0.000000,0.000000,1.192834,0.000000,0.105090,0.183116,0.166774,0.049618,0.097230,0.090801
2,HOBBIES_1_003_CA_1_validation,0.506919,0.551721,0.556527,0.541110,0.637841,0.832837,0.854774,0.578890,0.576483,0.434955,0.577135,0.777822,1.131037,0.808969,0.748165,0.556088,0.649401,0.532521,0.753691,0.858671,1.667238,0.706595,0.635462,0.562904,0.676679,0.844390,1.175036,1.048926
3,HOBBIES_1_004_CA_1_validation,1.330733,1.413427,1.687790,1.764875,1.595528,2.121065,4.741470,1.745639,0.989795,1.141659,1.555010,1.805885,1.974364,2.646898,1.715499,1.264823,0.960867,1.188757,1.478807,1.381116,4.923216,1.134674,0.969697,0.982375,1.379881,1.429551,2.746398,3.999021
4,HOBBIES_1_005_CA_1_validation,1.335066,1.340582,1.379598,1.336054,1.384215,0.657282,1.313109,1.643950,1.804935,1.485900,1.440515,1.594619,0.114099,1.664207,1.610674,1.560706,1.287365,1.183754,1.546010,0.970409,0.000000,1.646439,1.358292,1.164591,1.172624,1.490454,1.685852,2.548416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.510524,0.390425,0.472884,0.571214,0.613593,0.249343,0.540720,0.525568,0.513331,0.546845,0.609304,0.706948,0.635504,0.784078,0.624353,0.560856,0.623766,0.693965,0.834109,0.634540,0.918249,0.574352,0.523367,0.575044,0.643751,0.763619,0.403577,0.520292
60976,FOODS_3_824_WI_3_evaluation,0.365341,0.348874,0.292925,0.291059,0.170923,0.296711,0.291026,0.292942,0.348928,0.402216,0.360405,0.149901,0.247821,0.253369,0.258629,0.293681,0.389300,0.354910,0.132642,0.255410,0.012830,0.320340,0.293686,0.308581,0.299264,0.042666,0.116040,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.592966,0.508462,0.590797,0.545904,0.495142,0.515181,0.796410,0.861869,0.872277,0.837347,0.932802,0.877956,1.144232,1.127632,1.037094,0.781986,0.807892,0.797492,0.776183,0.950929,1.033215,0.712112,0.566986,0.527962,0.554274,0.510739,0.541289,0.585113
60978,FOODS_3_826_WI_3_evaluation,1.678722,1.586307,1.371468,1.203350,1.146666,1.466879,1.567255,1.678174,1.496696,1.549521,1.277129,1.426952,1.455104,1.725677,1.767920,1.673940,1.413700,1.196599,1.084073,1.300862,1.391108,1.568342,1.464760,1.217276,1.055087,0.904945,1.199577,1.665743
